# Otimização de Portfólio em BRL 
_Markowitz, Fronteira Eficiente, Monte Carlo e Sharpe vs CDI_

#### BLOCO 1 - Definição dos ativos

In [1]:
# ====== Configurações ======
DATA_INICIO = "2020-01-01"
DATA_FIM = "2026-01-01"
DIAS_UTEIS_ANO = 252  # referência (pregão)

# (Flag) Mistura ações/ETFs x cripto no calendário
# False: dias úteis (252) — simples; cripto no fim de semana é ignorado
# True: diário (365) — inclui fim de semana; ações ficam "flat" no fim de semana
USAR_CALENDARIO_DIARIO = True

FATOR_ANUALIZACAO = 365 if USAR_CALENDARIO_DIARIO else 252

# ====== Ativos (moeda-base: BRL) ======
# Brasil (B3) - já em BRL
ativos_br = {
    "PETR4": "PETR4.SA",
    "ITUB4": "ITUB4.SA",
#    "VALE3": "VALE3.SA",
#    "BBAS3": "BBAS3.SA",
#    "QBTC11": "QBTC11.SA",
    "KDIF11": "KDIF11.SA"
}

# EUA - em USD (serão convertidos para BRL)

ativos_us = {
#    "GOOGL": "GOOGL",
#    "NVDA": "NVDA",
    "NDAQ": "NDAQ",
#    "META": "META",
#    "AMZN": "AMZN",
#    "VOO": "VOO",
}


# Cripto - em USD (serão convertidos para BRL)

cripto_usd = {
#    "BTC": "BTC-USD",
#    "SOL": "SOL-USD",
}


# USDC = CAIXA (fora da otimização)
# Se quiser usar na carteira final como % em caixa, defina aqui:
PESO_CAIXA_USDC = 0.00  # ex.: 0.10 para 10% em caixa (USDC)
RETORNO_CAIXA_ANUAL = 0.0  # opcional: defina um retorno anual pro caixa (ex.: CDI aproximado)


#### Bloco 2 - Bibliotecas Necessárias

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize
# import matplotlib.pyplot as plt
import plotly.graph_objects as go

#### Etapa 1: Download + conversão USD→BRL + retornos e cov (substitua o seu bloco “Baixar dados / Calcular retornos”)

In [3]:
def _baixar_preco_adjclose(tickers, start, end):
    raw = yf.download(tickers, start=start, end=end, progress=False, auto_adjust=False)
    if raw.empty:
        return pd.DataFrame()

    # Preferir Adj Close, mas fazer fallback para Close
    if isinstance(raw.columns, pd.MultiIndex):
        if "Adj Close" in raw.columns.get_level_values(0):
            px = raw["Adj Close"].copy()
        elif "Close" in raw.columns.get_level_values(0):
            px = raw["Close"].copy()
        else:
            raise ValueError("Não encontrei colunas 'Adj Close' nem 'Close' no retorno do yfinance.")
    else:
        # Caso raro: sem MultiIndex
        px = raw.copy()

    px = px.dropna(axis=1, how="all")
    return px


def _baixar_usdbrl(start, end):
    # Tenta os dois formatos mais comuns no Yahoo
    for fx_ticker in ["BRL=X", "USDBRL=X"]:
        fx = _baixar_preco_adjclose([fx_ticker], start, end)
        if not fx.empty:
            s = fx.iloc[:, 0].rename("USDBRL").dropna()
            return s
    raise ValueError("Não consegui baixar o câmbio USD/BRL (tentei 'BRL=X' e 'USDBRL=X').")


# ====== Monta lista de tickers a baixar (USDC fora) ======
tickers_br = list(ativos_br.values())
tickers_us = list(ativos_us.values())
tickers_crypto = list(cripto_usd.values())

TICKERS = tickers_br + tickers_us + tickers_crypto

# ====== Baixar preços ======
precos_raw = _baixar_preco_adjclose(TICKERS, DATA_INICIO, DATA_FIM)
if precos_raw.empty:
    raise ValueError("Download retornou vazio. Verifique tickers, conexão e período.")

usdbrl_raw = _baixar_usdbrl(DATA_INICIO, DATA_FIM)

# ====== Calendário: dias úteis (252) vs diário (365) ======
if USAR_CALENDARIO_DIARIO:
    # calendário diário: inclui fins de semana
    idx = pd.date_range(start=precos_raw.index.min(), end=precos_raw.index.max(), freq="D")

    # Reindex e forward-fill: ações ficam constantes no fim de semana; cripto já tem dados diários
    precos = precos_raw.reindex(idx).ffill()
    usdbrl = usdbrl_raw.reindex(idx).ffill()

else:
    # dias úteis: usa apenas interseção das datas com USD/BRL (normalmente pregões)
    df_tmp = precos_raw.join(usdbrl_raw, how="inner")
    if df_tmp.empty:
        raise ValueError("Após alinhar com o câmbio (dias úteis), não restaram datas em comum.")
    # separa novamente
    usdbrl = df_tmp["USDBRL"]
    precos = df_tmp.drop(columns=["USDBRL"])

# Junta para converter USD -> BRL
df = precos.join(usdbrl.rename("USDBRL"), how="inner" if not USAR_CALENDARIO_DIARIO else "left").ffill()

# ====== Converter colunas USD -> BRL ======
usd_cols = [c for c in (tickers_us + tickers_crypto) if c in df.columns]
for c in usd_cols:
    df[c] = df[c] * df["USDBRL"]

# Remove coluna do câmbio do dataframe de preços
dados_brl = df.drop(columns=["USDBRL"])

# ====== Renomear colunas para os nomes “lógicos” (PETR4, NVDA, BTC, etc.) ======
ticker_to_nome = {v: k for k, v in ativos_br.items()}
ticker_to_nome.update({v: k for k, v in ativos_us.items()})
ticker_to_nome.update({v: k for k, v in cripto_usd.items()})

dados_brl = dados_brl.rename(columns=ticker_to_nome)

# Remove colunas eventualmente faltantes
dados_brl = dados_brl.dropna(axis=1, how="all")
if dados_brl.shape[1] < 2:
    raise ValueError("Poucos ativos com dados válidos após limpeza. Verifique tickers.")

# ====== Retornos ======
retornos = dados_brl.pct_change().dropna()

# ====== Estatísticas anualizadas (Markowitz) ======
retornos_medios_anuais = retornos.mean() * FATOR_ANUALIZACAO
matriz_covariancia = retornos.cov() * FATOR_ANUALIZACAO

print("Calendário diário:", USAR_CALENDARIO_DIARIO, "| Fator anualização:", FATOR_ANUALIZACAO)
print("Ativos usados na otimização (tudo em BRL):", list(retornos.columns))
print("PESO_CAIXA_USDC (fora da otimização):", PESO_CAIXA_USDC)


Calendário diário: True | Fator anualização: 365
Ativos usados na otimização (tudo em BRL): ['ITUB4', 'KDIF11', 'NDAQ', 'PETR4']
PESO_CAIXA_USDC (fora da otimização): 0.0


#### Etapa 1.5: Taxa livre de risco (rf) via CDI (B3/CETIP) usando BCData/SGS (série 12)


In [4]:
# =============================================================================
# Etapa 1.5 — Taxa livre de risco (CDI B3/CETIP)
# -----------------------------------------------------------------------------
# Observação importante:
# - O CDI (Taxa DI) é calculado/publicado oficialmente pela B3/CETIP.
# - Aqui usamos a API pública do BCB (SGS - série 12), que distribui o CDI diário.
#
# POR QUE seu print pode dar ~9-10% a.a.?
# - Se você anualizar a MÉDIA do CDI diário ao longo de 2020-2025, vai sair bem menor que o CDI "hoje"
#   (porque a Selic/CDI variaram bastante no período).
#
# RECOMENDAÇÃO:
# - Para Sharpe "de hoje" (uso mais comum na tomada de decisão), use o CDI ATUAL (último valor diário).
# - Para Sharpe histórico/in-sample, use a média composta do CDI no mesmo período dos retornos.
# =============================================================================

RF_MODO = "atual"          # "atual" | "media_periodo"
RF_FALLBACK = 0.10         # 10% a.a. se tudo falhar (ajuste se quiser)

def _cdi_diario_ultimo_via_sgs() -> float:
    """Retorna o último CDI diário (decimal/dia) via SGS série 12 (valor em % a.d.)."""
    url = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.12/dados/ultimos/1?formato=json"
    df = pd.read_json(url)

    if df.empty:
        raise ValueError("SGS retornou vazio para o último CDI (série 12).")

    # valor vem como string com vírgula brasileira em algumas rotas
    valor_str = str(df.loc[0, "valor"]).replace(",", ".")
    taxa_diaria_percentual = float(valor_str)   # ex: 0.0453 (% a.d.)
    taxa_diaria = taxa_diaria_percentual / 100.0
    return float(taxa_diaria)

def _cdi_diario_periodo_via_sgs(data_inicio: str, data_fim: str) -> pd.Series:
    """CDI (B3/CETIP) via SGS série 12 no período. Retorna em decimal/dia."""
    di = pd.to_datetime(data_inicio).strftime("%d/%m/%Y")
    df = pd.to_datetime(data_fim).strftime("%d/%m/%Y")

    url = (
        "https://api.bcb.gov.br/dados/serie/bcdata.sgs.12/dados"
        f"?formato=json&dataInicial={di}&dataFinal={df}"
    )

    cdi = pd.read_json(url)
    if cdi.empty:
        raise ValueError("CDI vazio. Verifique período e acesso à internet.")
    cdi["data"] = pd.to_datetime(cdi["data"], dayfirst=True)

    # 'valor' vem como % a.d. (pode vir com vírgula)
    cdi["valor"] = cdi["valor"].astype(str).str.replace(",", ".", regex=False)
    cdi["valor"] = pd.to_numeric(cdi["valor"], errors="coerce")

    s = (cdi.set_index("data")["valor"] / 100.0).rename("CDI_d")  # % a.d. -> decimal/dia
    return s.dropna()

def _anualizar_taxa_diaria_composta(taxa_diaria: float, dias_ano_ref: int = 252) -> float:
    """(1 + taxa_diária)^252 - 1"""
    return float((1.0 + taxa_diaria) ** dias_ano_ref - 1.0)

def _cdi_media_composta_anual(cdi_diario: pd.Series, dias_ano_ref: int = 252) -> float:
    """Anualiza usando média dos logs (equivalente composto)."""
    return float(np.expm1(np.log1p(cdi_diario).mean() * dias_ano_ref))

# --- Calcula rf ---
RF_ANUAL_ATUAL = np.nan
RF_ANUAL_MEDIA_PERIODO = np.nan

try:
    # CDI ATUAL (último diário) -> anualizado
    cdi_d_ult = _cdi_diario_ultimo_via_sgs()
    RF_ANUAL_ATUAL = _anualizar_taxa_diaria_composta(cdi_d_ult, dias_ano_ref=252)

    # CDI MÉDIO do período (opcional, mas útil para comparação/Sharpe histórico)
    cdi_d_periodo = _cdi_diario_periodo_via_sgs(DATA_INICIO, DATA_FIM)
    RF_ANUAL_MEDIA_PERIODO = _cdi_media_composta_anual(cdi_d_periodo, dias_ano_ref=252)

    if RF_MODO.lower() == "media_periodo":
        RF_ANUAL = float(RF_ANUAL_MEDIA_PERIODO)
    else:
        RF_ANUAL = float(RF_ANUAL_ATUAL)

except Exception as e:
    RF_ANUAL = float(RF_FALLBACK)
    print("⚠️ Não foi possível baixar CDI (usando fallback). Erro:", e)

print(f"RF_ANUAL_ATUAL (último CDI diário -> 252)........: {RF_ANUAL_ATUAL*100:.2f}% a.a.")
print(f"RF_ANUAL_MEDIA_PERIODO (média 2020-2025 -> 252): {RF_ANUAL_MEDIA_PERIODO*100:.2f}% a.a.")
print(f"RF_ANUAL USADO NO SHARPE (modo='{RF_MODO}').....: {RF_ANUAL*100:.2f}% a.a.")


RF_ANUAL_ATUAL (último CDI diário -> 252)........: 14.90% a.a.
RF_ANUAL_MEDIA_PERIODO (média 2020-2025 -> 252): 9.58% a.a.
RF_ANUAL USADO NO SHARPE (modo='atual').....: 14.90% a.a.


> **Nota sobre ações/ETFs x cripto (calendário):**  
> Seu notebook pode usar **252 dias úteis**, o que é coerente se você modela rebalanceamento apenas em pregão ou **“diário” (365)**
> Para cripto (BTC/SOL), usar 252 dias úteis, **ignora fins de semana** e pode **subestimar** a volatilidade.
>  
> **Alternativas ver Bloco 1 desse notebook: (Flag) Mistura ações/ETFs x cripto no calendário**  
> 1) **Modelo “dias úteis” (252):** mantém tudo em pregão (simples e consistente com bolsa). **USAR_CALENDARIO_DIARIO = False**
> 2) **Modelo “diário” (365):** reindexa tudo para todos os dias; ações/ETFs ficam “flat” no fim de semana (retorno 0), e cripto mantém variação 7/7. **USAR_CALENDARIO_DIARIO = True**



#### Etapa 2: Funções de Cálculo das Métricas

In [5]:
def calcular_metricas_portfolio(pesos, retornos_medios_anuais, matriz_covariancia, rf_anual=0.0):
    """
    Calcula retorno, volatilidade e Sharpe Ratio de um portfólio.

    Parâmetros:
        pesos: array numpy com os pesos de cada ativo
        retornos_medios_anuais: série pandas com retornos esperados (anualizados)
        matriz_covariancia: matriz de covariância anualizada
        rf_anual: taxa livre de risco anual (ex.: CDI), em decimal (0.10 = 10% a.a.)

    Retorna:
        array [retorno, volatilidade, sharpe_ratio]
    """
    pesos = np.array(pesos, dtype=float)

    # Retorno do portfólio: Rp = Σ(Wi × Ri)
    retorno_portfolio = float(np.sum(retornos_medios_anuais * pesos))

    # Volatilidade: σp = √(W^T × Σ × W)
    volatilidade_portfolio = float(np.sqrt(np.dot(pesos.T, np.dot(matriz_covariancia, pesos))))

    # Sharpe Ratio: (Rp - rf) / σp
    if volatilidade_portfolio > 0:
        sharpe_ratio = (retorno_portfolio - rf_anual) / volatilidade_portfolio
    else:
        sharpe_ratio = np.nan

    return np.array([retorno_portfolio, volatilidade_portfolio, sharpe_ratio])


#### Etapa 3: Funções Objetivo para Otimização

In [6]:
def minimizar_volatilidade(pesos, retornos_medios_anuais, matriz_covariancia):
    """Função objetivo para encontrar o portfólio de Mínima Variância."""
    return calcular_metricas_portfolio(pesos, retornos_medios_anuais, matriz_covariancia, rf_anual=RF_ANUAL)[1]

def maximizar_sharpe(pesos, retornos_medios_anuais, matriz_covariancia):
    """Função objetivo para encontrar o portfólio de Sharpe Máximo (excesso vs rf=CDI)."""
    return -calcular_metricas_portfolio(pesos, retornos_medios_anuais, matriz_covariancia, rf_anual=RF_ANUAL)[2]


#### Etapa 4: Configuração da Otimização

In [7]:
# Parâmetros (BUG FIX: dimensionar pelos dados realmente usados)
ativos_otimizacao = list(retornos.columns)
num_ativos = len(ativos_otimizacao)

pesos_iniciais = np.array([1.0/num_ativos] * num_ativos)  # distribuição igual

# Restrições e limites
limites = tuple((0, 1) for _ in range(num_ativos))               # cada peso entre 0% e 100%
restricoes = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0})  # soma = 1

# Otimização: Carteira de Mínima Variância
otimizacao_min_vol = minimize(
    minimizar_volatilidade,
    pesos_iniciais,
    args=(retornos_medios_anuais, matriz_covariancia),
    method='SLSQP',
    bounds=limites,
    constraints=restricoes
)

if not otimizacao_min_vol.success:
    print("⚠️ Otimização min vol não convergiu:", otimizacao_min_vol.message)

pesos_min_vol = otimizacao_min_vol.x
metricas_min_vol = calcular_metricas_portfolio(pesos_min_vol, retornos_medios_anuais, matriz_covariancia, rf_anual=RF_ANUAL)

# Otimização: Portfólio de Sharpe Máximo (vs CDI)
otimizacao_max_sharpe = minimize(
    maximizar_sharpe,
    pesos_iniciais,
    args=(retornos_medios_anuais, matriz_covariancia),
    method='SLSQP',
    bounds=limites,
    constraints=restricoes
)

if not otimizacao_max_sharpe.success:
    print("⚠️ Otimização max sharpe não convergiu:", otimizacao_max_sharpe.message)

pesos_max_sharpe = otimizacao_max_sharpe.x
metricas_max_sharpe = calcular_metricas_portfolio(pesos_max_sharpe, retornos_medios_anuais, matriz_covariancia, rf_anual=RF_ANUAL)

print("Carteira Mín Vol (ret, vol, sharpe):", metricas_min_vol)
print("Carteira Sharpe Máx (ret, vol, sharpe):", metricas_max_sharpe)


Carteira Mín Vol (ret, vol, sharpe): [0.17803597 0.1073928  0.27037335]
Carteira Sharpe Máx (ret, vol, sharpe): [0.3580657  0.1649635  1.26734633]


#### Etapa 5: Simulação de Monte Carlo para Visualizar a Fronteira

In [8]:
# Etapa 5: Simulação de Monte Carlo (amostragem melhor do espaço factível)

np.random.seed(42)

num_portfolios = 100000

def amostrar_pesos(n_assets, n_samples, alpha_div=1.0, alpha_conc=0.25, frac_conc=0.6, frac_cantos=0.1):
    """
    Gera pesos que representam melhor o espaço factível:
    - parte mais concentrada (alpha_conc < 1)
    - parte mais diversificada (alpha_div = 1)
    - injeta alguns pontos próximos aos 'cantos' (quase 1 ativo)
    """
    n_conc = int(n_samples * frac_conc)
    n_div = n_samples - n_conc

    w_conc = np.random.dirichlet([alpha_conc] * n_assets, size=n_conc)
    w_div  = np.random.dirichlet([alpha_div]  * n_assets, size=n_div)

    n_cantos = int(n_samples * frac_cantos)
    if n_cantos > 0:
        w_corner = np.zeros((n_cantos, n_assets))
        idx = np.random.randint(0, n_assets, size=n_cantos)
        for i in range(n_cantos):
            main = np.random.uniform(0.75, 0.98)
            rest = 1.0 - main
            tmp = np.random.dirichlet([alpha_conc] * n_assets)
            tmp[idx[i]] = 0.0
            tmp = tmp / tmp.sum()
            w_corner[i] = tmp * rest
            w_corner[i, idx[i]] = main
        W = np.vstack([w_conc, w_div, w_corner])
    else:
        W = np.vstack([w_conc, w_div])

    return W / W.sum(axis=1, keepdims=True)

W = amostrar_pesos(num_ativos, num_portfolios)

# Vetorizado (muito mais rápido que loop)
ret_arr = W @ retornos_medios_anuais.values
vol_arr = np.sqrt(np.einsum("ij,jk,ik->i", W, matriz_covariancia.values, W))
sharpe_arr = np.where(vol_arr > 0, (ret_arr - RF_ANUAL) / vol_arr, np.nan)

resultados = np.vstack([ret_arr, vol_arr, sharpe_arr])  # [retorno, risco, sharpe]


#### Etapa 6: Visualização da Fronteira Eficiente

In [9]:
# Etapa 6: Visualização (Plotly) — nuvem + fronteira eficiente "de verdade" + destaques
# Padrão de cores mantido: Viridis + estrela vermelha (min vol) + estrela verde (Sharpe máx)

def calcular_fronteira_eficiente(retornos_medios_anuais, matriz_covariancia, limites, n_pontos=120,
                              tol_vol=1e-6, tol_target=1e-4):
    """Fronteira eficiente (numérica, com restrições).

    Estratégia:
    1) Resolve min-variância para uma grade de retornos-alvo (SLSQP)
    2) Remove pontos dominados / duplicados em volatilidade (que causam 'zig-zag' no plot)
       mantendo o envelope superior (maior retorno para cada nível de risco).
    """
    mu_vec = retornos_medios_anuais.values.astype(float)
    cov_mat = matriz_covariancia.values.astype(float)

    r_min = float(mu_vec.min())
    r_max = float(mu_vec.max())
    alvos = np.linspace(r_min, r_max, n_pontos)

    pontos = []
    w0 = np.array([1.0/len(mu_vec)] * len(mu_vec))

    for alvo in alvos:
        cons = (
            {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0},
            {'type': 'eq', 'fun': lambda w, a=alvo: float(np.dot(w, mu_vec) - a)},
        )

        res = minimize(lambda w: float(w @ cov_mat @ w), w0, method='SLSQP', bounds=limites, constraints=cons)

        if res.success:
            w = res.x
            ret = float(np.dot(w, mu_vec))
            vol = float(np.sqrt(w @ cov_mat @ w))

            # garante que o solver realmente atingiu o retorno-alvo (tolerância)
            if abs(ret - alvo) <= max(tol_target, 1e-6):
                pontos.append((ret, vol))
                w0 = w  # warm start

    if not pontos:
        return np.array([]), np.array([])

    df = pd.DataFrame(pontos, columns=["ret", "vol"]).dropna()
    df = df[df["vol"] > 0].copy()

    # Dedup por volatilidade (bin) e pega o maior retorno por bin:
    df["vol_bin"] = (df["vol"] / tol_vol).round().astype(int)
    df = df.groupby("vol_bin", as_index=False).agg(vol=("vol", "mean"), ret=("ret", "max"))

    # Envelope superior: ao aumentar o risco, mantém apenas pontos que "batem" novo máximo de retorno
    df = df.sort_values("vol").reset_index(drop=True)
    df["ret_cummax"] = df["ret"].cummax()
    df = df[df["ret"] >= df["ret_cummax"] - 1e-12].copy()

    return df["ret"].to_numpy(), df["vol"].to_numpy()

front_ret, front_vol = calcular_fronteira_eficiente(retornos_medios_anuais, matriz_covariancia, limites, n_pontos=120)

fig = go.Figure()

# Nuvem de portfólios (coloridos pelo Sharpe)
fig.add_trace(
    go.Scatter(
        x=resultados[1, :] * 100,  # risco em %
        y=resultados[0, :] * 100,  # retorno em %
        mode="markers",
        name="Portfólios (Monte Carlo)",
        marker=dict(
            color=resultados[2, :],
            colorscale="Viridis",
            showscale=True,
            colorbar=dict(title="Índice de Sharpe (vs CDI)"),
            size=6,
            opacity=0.5,
            symbol="circle",
        ),
        hovertemplate=(
            "Volatilidade: %{x:.2f}%<br>"
            "Retorno: %{y:.2f}%<br>"
            "Sharpe: %{marker.color:.3f}"
            "<extra></extra>"
        ),
    )
)

# Fronteira eficiente (linha)
if len(front_ret) > 0:
    fig.add_trace(
        go.Scatter(
            x=front_vol * 100,
            y=front_ret * 100,
            mode="lines",
            name="Fronteira Eficiente (otimizada)",
            line=dict(width=3),
            hovertemplate="Volatilidade: %{x:.2f}%<br>Retorno: %{y:.2f}%<extra></extra>",
        )
    )

# Destaques
fig.add_trace(
    go.Scatter(
        x=[metricas_min_vol[1] * 100],
        y=[metricas_min_vol[0] * 100],
        mode="markers",
        name="Mínima Volatilidade",
        marker=dict(symbol="star", size=18, color="red", line=dict(color="black", width=2)),
        hovertemplate="<b>Mínima Volatilidade</b><br>Vol: %{x:.2f}%<br>Ret: %{y:.2f}%<extra></extra>",
    )
)

fig.add_trace(
    go.Scatter(
        x=[metricas_max_sharpe[1] * 100],
        y=[metricas_max_sharpe[0] * 100],
        mode="markers",
        name="Sharpe Máximo",
        marker=dict(symbol="star", size=18, color="green", line=dict(color="black", width=2)),
        hovertemplate="<b>Sharpe Máximo</b><br>Vol: %{x:.2f}%<br>Ret: %{y:.2f}%<extra></extra>",
    )
)

fig.update_layout(
    title=dict(text="Fronteira Eficiente (Markowitz) em BRL — Sharpe vs CDI", x=0.5),
    xaxis_title="Volatilidade Anualizada (Risco) — %",
    yaxis_title="Retorno Esperado Anualizado — %",
    template="plotly_white",
    legend=dict(x=0.01, y=0.99),
    width=950,
    height=650,
)

fig.update_xaxes(showgrid=True, gridwidth=1)
fig.update_yaxes(showgrid=True, gridwidth=1)

fig.show()


#### Etapa 7: Resumo e peso dos portifólios otmizados

In [10]:
df_metricas = pd.DataFrame([
    {
        "Carteira": "Mín. Volatilidade",
        "Retorno anual esperado": metricas_min_vol[0],
        "Volatilidade anual": metricas_min_vol[1],
        "Sharpe (rf=CDI)": metricas_min_vol[2],
    },
    {
        "Carteira": "Sharpe Máximo",
        "Retorno anual esperado": metricas_max_sharpe[0],
        "Volatilidade anual": metricas_max_sharpe[1],
        "Sharpe (rf=CDI)": metricas_max_sharpe[2],
    },
])

# versão só para exibição (strings)
df_metricas_view = df_metricas.copy()
df_metricas_view["Retorno anual esperado"] = df_metricas_view["Retorno anual esperado"].map(lambda x: f"{x:.2%}")
df_metricas_view["Volatilidade anual"] = df_metricas_view["Volatilidade anual"].map(lambda x: f"{x:.2%}")
df_metricas_view["Sharpe (rf=CDI)"] = df_metricas_view["Sharpe (rf=CDI)"].map(lambda x: f"{x:.2f}")

In [11]:
# =========================
# PESOS — DataFrame (wide)
# =========================
df_pesos = pd.DataFrame({
    "Ativo": ativos_otimizacao,
    "Peso_Min_Vol": pesos_min_vol,
    "Peso_Max_Sharpe": pesos_max_sharpe,
})##.set_index("Ativo")

# (Opcional) versão só para exibição (% como string)
df_pesos_view = df_pesos.copy()
# começa da segunda coluna porque a primeira ('Ativo') é texto
for col in df_pesos_view.columns[1:]:
    df_pesos_view[col] = df_pesos_view[col].map(lambda x: f"{x:.2%}")

'''
# =========================
# PESOS — DataFrame (long)  [recomendado para Plotly]
# =========================
df_pesos_long = df_pesos.reset_index().melt(
    id_vars="Ativo",
    var_name="Carteira",
    value_name="Peso"
)

# versão só para exibição
df_pesos_long_view = df_pesos_long.copy()
df_pesos_long_view["Peso"] = df_pesos_long_view["Peso"].map(lambda x: f"{x:.2%}")

display(df_pesos_long_view)
'''

# =========================
# TOP N pesos por carteira (útil p/ leitura rápida)
# =========================
TOP_N = 15

def top_n(df_wide, col, n=TOP_N):
    out = df_wide[[col]].sort_values(col, ascending=False).head(n).copy()
    out[col] = out[col].map(lambda x: f"{x:.2%}")
    out.columns = [f"Top {n} - {col}"]
    return out

'''
# =========================
# (Opcional) Checagens rápidas
# =========================
print("Soma dos pesos (Min Vol):", df_pesos["Peso_Min_Vol"].sum())
print("Soma dos pesos (Max Sharpe):", df_pesos["Peso_Max_Sharpe"].sum())
'''

'\n# =========================\n# (Opcional) Checagens rápidas\n# =========================\nprint("Soma dos pesos (Min Vol):", df_pesos["Peso_Min_Vol"].sum())\nprint("Soma dos pesos (Max Sharpe):", df_pesos["Peso_Max_Sharpe"].sum())\n'

In [12]:
# Metricas das carteiras otimizadas
print("\nMétricas das carteiras otimizadas:")
display(df_metricas_view)

# =========================

# PESOS — DataFrame (wide)
print("\nPesos das carteiras otimizadas:")
display(df_pesos_view)

# =========================

# TOP N pesos por carteira (útil p/ leitura rápida)
print(f"\nTop {TOP_N} pesos por carteira:")
display(top_n(df_pesos, "Peso_Min_Vol", TOP_N))
display(top_n(df_pesos, "Peso_Max_Sharpe", TOP_N))


Métricas das carteiras otimizadas:


,Carteira,Retorno anual esperado,Volatilidade anual,Sharpe (rf=CDI)
0,Mín. Volatilidade,17.80%,10.74%,0.27
1,Sharpe Máximo,35.81%,16.50%,1.27



Pesos das carteiras otimizadas:


,Ativo,Peso_Min_Vol,Peso_Max_Sharpe
0,ITUB4,21.00%,58.06%
1,KDIF11,50.12%,0.00%
2,NDAQ,12.34%,41.94%
3,PETR4,16.54%,0.00%



Top 15 pesos por carteira:


,Top 15 - Peso_Min_Vol
1,50.12%
0,21.00%
3,16.54%
2,12.34%


,Top 15 - Peso_Max_Sharpe
0,58.06%
2,41.94%
1,0.00%
3,0.00%


#### Etapa 8: Visual da matriz de covariância (Plotly Heatmap: frio/quente)

In [13]:
# Visualização da Matriz de Covariância (Plotly) - cores frias/quentes conforme sinal e magnitude
# Como o gráfico principal está em %, aqui mostramos a covariância em (%²/ano)
cov_perc2 = matriz_covariancia.copy() * (100**2)

fig_cov = go.Figure(
    data=go.Heatmap(
        z=cov_perc2.values,
        x=list(cov_perc2.columns),
        y=list(cov_perc2.index),
        colorscale="RdBu",   # azul (negativo) ↔ vermelho (positivo)
        zmid=0,              # centraliza em 0 (essencial p/ quente/frio)
        colorbar=dict(title="Covariância<br>(%²/ano)"),
        hovertemplate="Ativo X: %{x}<br>Ativo Y: %{y}<br>Cov: %{z:.4f} (%²/ano)<extra></extra>",
    )
)

fig_cov.update_layout(
    title=dict(text="Matriz de Covariância Anualizada (%²/ano)", x=0.5),
    template="plotly_white",
    width=800,
    height=680,
)

fig_cov.update_xaxes(side="top")
fig_cov.show()
